
<div hidden>

$\gdef\dd{\mathrm{d}}$

</div>

<div hidden>

$\gdef\abs#1{\left\vert#1\right\vert}$

</div>

<div hidden>

$\gdef\ve#1{\bm{#1}}$
</div>

<div hidden>

$\gdef\mat#1{\mathbf{#1}}$
</div>

<div hidden>
Authors: Felipe Uribe and Amal Alghamdi
</div>

# Two forward models

We introduce here two forward models: 1D Poisson and 1D convolution. We will use these forward models in the book to apply and demonstrate various concepts in Bayesian inversion.


## <font color=#CD853F> Contents of this notebook: </font>
  * [Learning objectives](#r-learning-objectives)
  * [Forward model: 1D Poisson](#r-forward-model-1d-poisson)
  * [Forward model: 1D convolution](#r-forward-model-1d-convolution)


## <font color=#CD853F> Learning objectives: </font> <a name="r-learning-objectives"></a>
  * Create (or load pre-existing) linear and non-linear forward models in CUQIpy and use them
  * Create and plot input for the forward models and compute and plot the corresponding output
  * Mathematically define two forward models: 1D Poisson and 1D Convolution and use them in CUQIpy
  

In [ ]:
from cuqi.testproblem import Poisson1D, Deconvolution1D, Heat1D
from cuqi.distribution import Gaussian
from cuqi.array import CUQIarray
import numpy as np
import matplotlib.pyplot as plt

## <font color=#CD853F> Forward model: 1D Poisson </font> <a name="r-forward-model-1d-poisson"></a>

Consider a heat conductive rod of length $L = \pi$ with a varying conductivity (the conductivity of the rod changes from point to point). We fix the temperature at the end-points of the rod and apply a heat source distributed along the length of the rod. We wait until the rod reaches an equilibrium temperature distribution. The equilibrium temperature of the rod is modelled using the second order steady-state PDE as

$$
\left\{
\begin{aligned}
& \dfrac{\dd}{\dd \xi}\left(u(\xi) \dfrac{\dd y(\xi)}{\dd \xi}\right) = -f(\xi), \quad & \xi\in (0,L) \\
& y(0) = y(L) = 0.
\end{aligned}
\right.
$$
Here, $y$ represents the temperature distribution along the rod, $u(\xi) $ is the unknown conductivity of the rod and $f(\xi)$ is a deterministic heat source given by

$$
\begin{aligned}
	f(\xi) = 10\exp( -\frac{ (\xi - L/2)^2} {0.02} ).
\end{aligned}
$$

To ensure that the conductivity of the rod is non-negative, we parameterize $u$ by a random variable $x$ as follows:
 
$$
 \begin{aligned}
 u( \cdot  ) = \exp( x( \cdot  ) )
 \end{aligned}
$$
where $x$ is not necessarily positive.

Let us load the forward model that maps the random variable $x$ to the temperature distribution $y$ in CUQIpy. We will use the following parameters:
* `dim` : number of discretization points for the rod
* `L` : length of the rod
* `f` : a function that represents the heat source

In [ ]:
dim = 128
L = np.pi
f = lambda xi: 10*np.exp(-(xi-L/2)**2 / 0.02)

Then we can load the 1D Poisson forward model as follows:

In [ ]:

A = Poisson1D(dim=dim, endpoint=L, source=f).model

We print the forward model to see its details.

In [ ]:
A

We can look at the domain and range geometries of the forward model.

In [ ]:
print(A.domain_geometry)
print(A.range_geometry)

These geometries are of type `Continuous1D` which represents a 1D continuous signal/field defined on a grid. We can view the grid:

In [ ]:
print(A.domain_geometry.grid)

Additionally, the properties `domain_dim` and `range_dim` of the forward model represent the dimension of the input and output of the forward model, respectively.

In [ ]:
print(A.domain_dim)
print(A.range_dim)

Let us create an array representing a constant conductivity over the grid nodes 

In [ ]:
some_x_array = 20*np.ones(A.domain_dim)

We can wrap the array in a `CUQIarray` object which is the main data structure in CUQIpy for variables (e.g. arrays and fields)

In [ ]:
some_x = CUQIarray(some_x_array, geometry=A.domain_geometry)

Note that we pass `geometry=A.domain_geometry` to equip the `CUQIarray` object with the same geometry as the domain of the forward model, which is has information about what the values of the array represent (e.g. function values on a grid for this case).

We can plot the conductivity array using the `plot` method of the `CUQIarray` object

In [ ]:
some_x.plot()

We can also evaluate the forward model at the conductivity array and plot the solution:

In [ ]:
some_y = A(some_x)
some_y.plot()

##### <font color=#8B4513> Exercise: </font>
  * Is this true for you: **"I can, mathematically, write a definition of a 1D Poisson forward model that maps the conductivity field to the temperature distribution field"**? If not, what questions do you have?
  * Trying a different conductivity profile: 
      * Create another constant conductivity profile `another_x` of value 20 as a `CUQIarray` object.
      * Evaluate the forward model at the new conductivity profile and store the result in a variable `another_y`. Plot the solution.
      * In one plot, compare the solutions `some_y` and `another_y` using the `plot` method of the `CUQIarray` object. What do you observe? and why?
  * Experimenting with setting up the `map` in `Poisson1D`:
      * Execute `help(Poisson1D)`
      * Note the `map` parameter of `Poisson1D`. We can use this parameter to transform the conductivity field before solving the PDE. Create the forward model again, name it `A_map`, with setting up `map` to be `lambda x: np.exp(x)` to ensure that the conductivity is always positive.
      * Inspect the domain and range geometries of the new forward model `A_map` what is different this time, and why?
      * Create a CUQIarray object `x_with_map` with a constant conductivity profile of value 0 and evaluate the forward model at `x_with_map` and store it in variable `y_with_map`. Make sure to use the right geometry for `x_with_map`.
      * Plot `x_with_map` using `x_with_map.plot()`, What do you observe?
      * Plot the solution `y_with_map` and `some_y` in the same plot. What do you observe? and why?
  * Experimenting with setting up the `field_type` in `Poisson1D`:
      * Note the `field_type` parameter of `Poisson1D`. We can use this parameter to change the parameterization of the conductivity field. Create the forward model again, name it `A_step`, with setting both the `map` as in the previous exercise and the `field_type` to be `"Step"` to generate a step parameterization of the conductivity field.
      * What is the domain and range geometries of the new forward model `A_step`?
      * Create `x_step` to be `x_step = CUQIarray(np.arange(A_step.domain_dim)*0.1, geometry=A_step.domain_geometry)`.
      * Plot `x_step` using `x_step.plot()`.
      * What does the dimension `A_step.domain_dim` represent? and is it equal to the size of the domain geometry grid?
      * Evaluate the forward model at `x_step` and store it in variable `y_step`. Plot the solution.
  * Is this true for you: **"I can load the pre-existing Poisson 1D forward model in CUQIpy and evaluate it on some input and visualize the input and output"**? If not, what questions do you have?

In [ ]:
# your code here

## <font color=#CD853F> Forward model: 1D convolution </font> <a name="r-forward-model-1d-convolution"></a>


The mathematical model for convolution of a random signal on a one-dimensional spatial domain $D$, can be written as a stochastic Fredholm integral equation of the first kind:

$$
\begin{aligned}
y(\xi) = \int_{D} k(\xi,\xi')x(\xi')\,\dd \xi' 
\end{aligned}
$$

where $y(\xi)$ denotes the convolved signal and we assume a Gaussian convolution kernel. In practice, a finite-dimensional representation of the integral equation is employed. After discretizing the signal domain into $N$ components, the convolution model can be expressed as a system of linear algebraic equations $\ve{y}=\mat{K}\ve{x}$. 

Let us load the forward model that maps the random variable $\ve{x}$ to the convolved signal $\ve{y}$ in CUQIpy. We will use the following parameters to specify the forward model:
* `dim` : is the number of discretization points for the signal, $N$
* `PSF` : a function that represents the point spread function, i.e., the convolution kernel
* `PSF_size` : the size of the PSF (less or equal to `dim`)
* `PSF_param` : A parameter of the PSF, the larger the size the more the blur applied to the signal
* `BC` : boundary conditions for the convolution

We set the parameters as follows:

In [ ]:
dim = 201
PSF = 'gauss' # Gaussian PSF
PSF_size = np.round(dim/3)
PSF_param = np.round(dim/20)
BC='reflect' # Boundary condition

Then we can load the 1D convolution forward model as follows (note that we refer to the forward model as a convolution model which we obtain from the `Deconvolution1D` test problem, the latter represents the BIP of deconvolving the signal): 

In [ ]:
A_deconv = Deconvolution1D(dim=dim, PSF=PSF, PSF_size=PSF_size, PSF_param=PSF_param, BC=BC).model

Let us create a function representing a signal that we want to convolve.

In [ ]:
signal_function = lambda xi: (xi>np.round(dim*4/10))*(xi<np.round(dim*6/10))

We evaluate the function `signal_function` at the discretization grid points and wrap it in a `CUQIarray` object.

In [ ]:
signal_array = signal_function(A_deconv.domain_geometry.grid)
signal = CUQIarray(signal_array, geometry=A_deconv.domain_geometry)

Let us plot the signal

In [ ]:
signal.plot()

Now, we evaluate the forward model at the signal and plot the solution:

In [ ]:
A_deconv(signal).plot()

We see that the signal is blurred due to applying the convolution operation (the forward model).

##### <font color=#8B4513> Exercise: </font>

- Type `help(Deconvolution1D)` to see the documentation of this test problem.
- Experimenting with the convolution strength:
    * Create another convolution model `B` with the same parameters as `A_deconv` but with a different `PSF_param` value, use `PSF_param=PSF_param/2`. Evaluate the forward model `B` at the signal and plot the solution along with `A_deconv(signal)` in the same plot. What do you observe?
- Experimenting with setting up the `PSF`:
    * Create a new convolution model `C` with the same parameters as `A_deconv` but with a different `PSF`, use `PSF=np.ones(m)/m`. Evaluate the forward model `C` at the signal and plot the solution along with `A_deconv(signal)` in the same plot (try for different values of m: 5, 10, 20). What do you observe?
- Experimenting with setting up the `BC`:
    * Create a new convolution model `D` with the same parameters as `A_deconv` but with a different `BC`, use `BC="periodic"`. Evaluate the forward model `D` at the signal and plot the solution along with `A_deconv(signal)` in the same plot. What do you observe?
    * Now create another signal `another_signal` as `another_signal = signal + 0.1*np.exp((D.domain_geometry.grid - np.round(dim*7/8))/10)`. Plot the new signal. Evaluate the forward models `A_deconv`, and `D` at the new signal and plot the solutions in the same plot. What do you observe?
* Is this true for you: **"I can load the pre-existing convolution 1D forward model in CUQIpy, experiment with its parameter setup, evaluate it on some input, and visualize the input and output"**? If not, what questions do you have?

In [ ]:
# your code here